In [ ]:
import requests
import json
import pandas as pd

def retrieve_latest_drugs(from_date, to_date, n_results=1000):
    url = 'https://api.fda.gov/drug/label.json'
    params = {
        'search': f'effective_time:[{from_date} TO {to_date}] AND version:1 AND openfda.product_type:human*',
        'limit': n_results
    }

    #getting the code to fine-tune the model
    response_columbia = response_columbia.get(url, params=params)
    print("Request URL:", respresponse_columbiaonse.url)
    print("Status Code:", response_columbia.status_code)

    #This was the main issue we encountered
    #so creating this to remove the issue
    if response_columbia.status_code == 200:
        try:
            #getting the response in json file
            fda_data_columbia_final_project = response_columbia.json()
            #printing out the available keys
            print("Keys:", fda_data_columbia_final_project.keys())
            return fda_data_columbia_final_project
        except json.JSONDecodeError:
            return None
    else:
        #if something goes wrong the section below is for debugging
        print(f"Error: {response_columbia.status_code}")
        print("Response Content:", response_columbia.text)
        return None


#trying the results with custom dates
#to see if it works
result = retrieve_latest_drugs("20241101", "20241115")

if result is not None:
    # Extract relevant data from the JSON response
    drug_data_columbia = result.get('results', [])
    print("Result:", json.dumps(result, indent=2))

    #Convert the drug data into a DataFrame
    #drug_df is our main dataframe
    drug_data_columbia_df = pd.DataFrame(drug_data_columbia)

    #if 'openfda' column exists and is not empty
    if 'openfda' in drug_data_columbia_df.columns and not drug_data_columbia_df['openfda'].isnull().all():
        # Expand the 'openfda' dictionary into multiple columns
        #to get the drugs from the dataset
        openfda_df_columbia = drug_data_columbia_df['openfda'].apply(lambda x: x if isinstance(x, dict) else {})
        #expanding nested dictinaries
        openfda_expanded_columbia = pd.json_normalize(openfda_df_columbia)
        openfda_expanded_columbia = openfda_expanded_columbia.add_prefix('openfda__')

        #Merge the expanded columns back into the original DataFrame
        drug_df_columbia_final_project = pd.concat([drug_df.drop('openfda', axis=1), openfda_expanded_columbia], axis=1)
    else:
        # If there's no openfda information or the column is empty, just proceed
        print("No 'openfda' data found or it's empty.")

    print(drug_df_columbia_final_project.head())
else:
    print("No data retrieved from the FDA API.")


Streaming output truncated to the last 5000 lines.
      "overdosage": [
        "10 OVERDOSAGE Significant lethality was observed in mice and rats after oral administration of 1000 mg/kg and 2000 mg/kg, respectively, about 44 and 170 times the maximum recommended human dose on a mg/m 2 basis. Limited data are available in regard to overdosage in humans. The most likely manifestation of overdosage would be hypotension and tachycardia; bradycardia could occur from parasympathetic (vagal) stimulation. If symptomatic hypotension should occur, supportive treatment should be instituted. Neither losartan nor its active metabolite can be removed by hemodialysis."
      ],
      "clinical_pharmacology": [
        "12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Action Angiotensin II [formed from angiotensin I in a reaction catalyzed by angiotensin converting enzyme (ACE, kininase II)] is a potent vasoconstrictor, the primary vasoactive hormone of the renin-angiotensin system, and an important compo

In [ ]:
import requests
import json
import pandas as pd

#defining the url
#this url will be the direct link to fdas webiste to fetch the data
#we are going to retrieve the latest drugs with the functions below

def retrieve_latest_drugs(from_date, to_date, n_results=1000):
    url = 'https://api.fda.gov/drug/label.json'
    params = {
        'search': f'effective_time:[{from_date} TO {to_date}] AND version:1 AND openfda.product_type:human*',
        'limit': n_results
    }

    #getting the code to fine-tune the model
    #first we need to make a hhtp get request to access the api
    #we are going to first check the request irl
    #if the request api is applicable then we are going to fetch from that website
    response_columbia = response_columbia.get(url, params=params)
    #This was the main issue we encountered
    #so creating this to remove the issue
    #checking if the response is successfull
    if response_columbia.status_code == 200:
        try:
            #getting the response in json file
            #parsing the json reposne
            fda_data_columbia_final_project = response_columbia.json()
            #printing out the available keys
            print("Keys:", fda_data_columbia_final_project.keys())
            return fda_data_columbia_final_project
        except json.JSONDecodeError:
            return None
    else:
        #if something goes wrong the section below is for debugging
        return None


#trying the results with custom dates
#to see if it works
result = retrieve_latest_drugs("20241101", "20241115")

if result is not None:
    #Extracting the relevant data from the JSON response
    #first we are going to list the drug entries
    #and then print the formatted JSON
    drug_data_columbia = result.get('results', [])
    print("Result:", json.dumps(result, indent=2))

    #Convert the drug data into a DataFrame
    #drug_df is our main dataframe
    #datafrane
    drug_data_columbia_df = pd.DataFrame(drug_data_columbia)

    #if 'openfda' column exists and is not empty
    if 'openfda' in drug_data_columbia_df.columns and not drug_data_columbia_df['openfda'].isnull().all():
        # Expand the 'openfda' dictionary into multiple columns
        #to get the drugs from the dataset
        #Expanding nested 'openfda' dictionaries into individual columns
        openfda_df_columbia = drug_data_columbia_df['openfda'].apply(lambda x: x if isinstance(x, dict) else {})
        #expanding nested dictinaries
        openfda_expanded_columbia = pd.json_normalize(openfda_df_columbia)
        openfda_expanded_columbia = openfda_expanded_columbia.add_prefix('openfda__')

        #Merging the expanded columns back into the original DataFrame
        #after that we have the concated version of the dataframe
        drug_df_columbia_final_project = pd.concat([drug_df.drop('openfda', axis=1), openfda_expanded_columbia], axis=1)
    else:
        # If there's no openfda information or the column is empty, just proceed
        print("No 'openfda' data found or it's empty.")

    #to observe better we are printing out the drugs list
    print(drug_df_columbia_final_project.head())
else:
    print("No data retrieved from the FDA API.")

#Creating a dataset that has both the biosimilars and dataset
def create_biosimilar_dataset(drug_df_columbia):
    if 'openfda__brand_name' not in drug_df_columbia.columns or 'openfda__generic_name' not in drug_df_columbia.columns:
        print("Required columns not found in DataFrame.")
        return pd.DataFrame()

    #a single value extraction
    def extract_first_value_columbia(val):
        if isinstance(val, list) and len(val) > 0:
            return val[0]
        return val

    #getting the biosimilar name and the openfda drug name
    #applying the function to every column
    drug_df_columbia['biosimilar_name'] = drug_df_columbia['openfda__brand_name'].apply(extract_first_value_columbia)
    #getting the reference product for the dataset
    drug_df_columbia['reference_product'] = drug_df_columbia['openfda__generic_name'].apply(extract_first_value_columbia)

    #filtering the rows so only biosimilar and reference product will be here so that we can
    #finetune
    subset_columbia_final = drug_df[['biosimilar_name', 'reference_product']].dropna().reset_index(drop=True)
    return subset_columbia_final


if result_columbia is not None:
    drug_data_columbia_final_project = result_columbia.get('results', [])
    drug_data_columbia_final_project = pd.DataFrame(drug_data_columbia_final_project)

    #Check if 'openfda' column exists and is a dict
    if 'openfda' in drudrug_data_columbia_final_projectg_df.columns and not drug_data_columbia_final_project['openfda'].isnull().all():
        #Expand the 'openfda' dictionary into multiple columns
        openfda_expanded_columbia_final = pd.json_normalize(drug_data_columbia_final_project['openfda'].apply(lambda x: x if isinstance(x, dict) else {}))
        openfda_expanded_columbia_final = openfda_expanded_columbia_final.add_prefix('openfda__')

        # Merge the expanded columns back into the original DataFrame
        drug_data_columbia_final_project = pd.concat([drug_data_columbia_final_project.drop('openfda', axis=1), openfda_expanded_columbia_final], axis=1)
    else:

    #now creating a biosimilar dataset
    biosimilar_dataset = create_biosimilar_dataset(drug_df)
    #printing the dataset
    print(biosimilar_dataset)
else:



Request URL: https://api.fda.gov/drug/label.json?search=effective_time%3A%5B20241101+TO+20241115%5D+AND+version%3A1+AND+openfda.product_type%3Ahuman%2A&limit=100&skip=0
Status Code: 200
Request URL: https://api.fda.gov/drug/label.json?search=effective_time%3A%5B20241101+TO+20241115%5D+AND+version%3A1+AND+openfda.product_type%3Ahuman%2A&limit=100&skip=100
Status Code: 200
Request URL: https://api.fda.gov/drug/label.json?search=effective_time%3A%5B20241101+TO+20241115%5D+AND+version%3A1+AND+openfda.product_type%3Ahuman%2A&limit=100&skip=200
Status Code: 200
Request URL: https://api.fda.gov/drug/label.json?search=effective_time%3A%5B20241101+TO+20241115%5D+AND+version%3A1+AND+openfda.product_type%3Ahuman%2A&limit=100&skip=300
Status Code: 200
Request URL: https://api.fda.gov/drug/label.json?search=effective_time%3A%5B20241101+TO+20241115%5D+AND+version%3A1+AND+openfda.product_type%3Ahuman%2A&limit=100&skip=400
Status Code: 200
Request URL: https://api.fda.gov/drug/label.json?search=effect

In [ ]:
import requests
import json
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Function to retrieve the latest drugs from the FDA API within a specific date range
def retrieve_latest_drugs_columbia(from_date, to_date, n_results=1000):
    url = 'https://api.fda.gov/drug/label.json'
    params = {
        'search': f'effective_time:[{from_date} TO {to_date}] AND version:1 AND openfda.product_type:human*',
        'limit': n_results
    }

    #getting the code to fine-tune the model
    #first we need to make a hhtp get request to access the api
    #we are going to first check the request irl
    #if the request api is applicable then we are going to fetch from that website
    response_columbia = requests.get(url, params=params)
    #This was the main issue we encountered
    if response_columbia.status_code == 200:
        try:
            #getting the response in json file
            #parsing the json reposne
            fda_data_columbia_final_project = response_columbia.json()
            #printing out the available keys
            print("Keys:", fda_data_columbia_final_project.keys())
            return fda_data_columbia_final_project
        except json.JSONDecodeError:
            return None
    else:
        #if something goes wrong the section below is for debugging
        return None

# Test the API function with a specific date range
result_columbia = retrieve_latest_drugs_columbia("20241101", "20241115")

if result_columbia is not None:
    #Extracting the relevant data from the JSON response
    #first we are going to list the drug entries
    #and then print the formatted JSON
    drug_data_columbia = result_columbia.get('results', [])
    print("Result:", json.dumps(result_columbia, indent=2))

    #Convert the drug data into a DataFrame
    #drug_df is our main dataframe
    drug_data_columbia_df = pd.DataFrame(drug_data_columbia)

    #if 'openfda' column exists and is not empty
    if 'openfda' in drug_data_columbia_df.columns and not drug_data_columbia_df['openfda'].isnull().all():
        # Expand the 'openfda' dictionary into multiple columns
        #to get the drugs from the dataset
        #Expanding nested 'openfda' dictionaries into individual columns
        openfda_df_columbia = drug_data_columbia_df['openfda'].apply(lambda x: x if isinstance(x, dict) else {})
        #expanding nested dictinaries
        openfda_expanded_columbia = pd.json_normalize(openfda_df_columbia)
        openfda_expanded_columbia = openfda_expanded_columbia.add_prefix('openfda__')

        #Merging the expanded columns back into the original DataFrame
        #after that we have the concated version of the dataframe
        drug_df_columbia_final_project = pd.concat([drug_data_columbia_df.drop('openfda', axis=1), openfda_expanded_columbia], axis=1)
    else:
        # If there's no openfda information or the column is empty, just proceed
        print("No 'openfda' data found or it's empty.")

    #to observe better we are printing out the drugs list
    print(drug_df_columbia_final_project.head())
else:
    print("No data retrieved from the FDA API.")


#Creating a dataset that has both the biosimilars and dataset
def create_biosimilar_dataset_columbia(drug_df_columbia):
    if 'openfda__brand_name' not in drug_df_columbia.columns or 'openfda__generic_name' not in drug_df_columbia.columns:
        print("Required columns not found in DataFrame.")
        return pd.DataFrame()

    #a single value extraction
    def extract_first_value_columbia(val):
        if isinstance(val, list) and len(val) > 0:
            return val[0]
        return val

    #getting the biosimilar name and the openfda drug name
    #applying the function to every column
    drug_df_columbia['biosimilar_name'] = drug_df_columbia['openfda__brand_name'].apply(extract_first_value_columbia)
    #getting the reference product for the dataset
    drug_df_columbia['reference_product'] = drug_df_columbia['openfda__generic_name'].apply(extract_first_value_columbia)

    #filtering the rows so only biosimilar and reference product will be here so that we can
    #finetune
    subset_columbia_final = drug_df_columbia[['biosimilar_name', 'reference_product']].dropna().reset_index(drop=True)
    return subset_columbia_final

if result_columbia is not None:
    drug_data_columbia_final_project = result_columbia.get('results', [])
    drug_data_columbia_final_project_df = pd.DataFrame(drug_data_columbia_final_project)

    #Check if 'openfda' column exists and is a dict
    if 'openfda' in drug_data_columbia_final_project_df.columns and not drug_data_columbia_final_project_df['openfda'].isnull().all():
        #Expand the 'openfda' dictionary into multiple columns
        openfda_expanded_columbia_final = pd.json_normalize(drug_data_columbia_final_project_df['openfda'].apply(lambda x: x if isinstance(x, dict) else {}))
        openfda_expanded_columbia_final = openfda_expanded_columbia_final.add_prefix('openfda__')

        #Merging the expanded columns back into the original DataFrame
        drug_data_columbia_final_project_df = pd.concat([drug_data_columbia_final_project_df.drop('openfda', axis=1), openfda_expanded_columbia_final], axis=1)

    #now creating a biosimilar dataset
    biosimilar_dataset_columbia = create_biosimilar_dataset_columbia(drug_data_columbia_final_project_df)
    #printing the dataset
    print(biosimilar_dataset_columbia)
else:
    print("No data retrieved from the FDA API.")

# Prompt: "Given a biosimilar name, determine the reference product.\nBiosimilar: {biosimilar_name}\nAnswer:"
# Target: {reference_product}

def format_example_columbia(row):
    instruction = "Given a biosimilar name, determine the reference product."
    prompt = f"Instruction: {instruction}\nBiosimilar: {row['biosimilar_name']}\nAnswer:"
    return {"input_text": prompt, "target_text": row["reference_product"]}

formatted_data_columbia = biosimilar_dataset_columbia.apply(format_example_columbia, axis=1)
formatted_data_columbia = list(formatted_data_columbia)  # Convert to list of dicts
hf_dataset_columbia_final = Dataset.from_list(formatted_data_columbia)

#splitting the dataset into train and test
split_dataset_columbia = hf_dataset_columbia_final.train_test_split(test_size=0.1)
train_dataset_columbia_med = split_dataset_columbia["train"]
test_dataset_columbia_med = split_dataset_columbia["test"]

#we are using the same model that we are using for our evaluations
#these codes are the same as the previous ones
model_name_columbia = "ruslanmv/Medical-Llama3-8B"
device_map_columbia = 'auto'

bnb_config_columbia = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model_columbia = AutoModelForCausalLM.from_pretrained(
    model_name_columbia,
    quantization_config=bnb_config_columbia,
    trust_remote_code=True,
    use_cache=False,
    device_map=device_map_columbia
)

tokenizer_columbia = AutoTokenizer.from_pretrained(model_name_columbia, trust_remote_code=True)
tokenizer_columbia.pad_token = tokenizer_columbia.eos_token

model_columbia = prepare_model_for_kbit_training(model_columbia)

lora_config_columbia = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model_columbia = get_peft_model(model_columbia, lora_config_columbia)

def tokenize_function_columbia(examples):
    #to tokenize we will combine all the feature
    full_text_columbia = [inp + " " + ans for inp, ans in zip(examples["input_text"], examples["target_text"])]
    return tokenizer_columbia(full_text_columbia, truncation=True, padding="max_length", max_length=512)

tokenized_train_columbia = train_dataset_columbia_med.map(tokenize_function_columbia, batched=True, remove_columns=train_dataset_columbia_med.column_names)
tokenized_eval_columbia = test_dataset_columbia_med.map(tokenize_function_columbia, batched=True, remove_columns=test_dataset_columbia_med.column_names)

data_collator_columbia = DataCollatorForLanguageModeling(tokenizer_columbia, mlm=False)


training_args_columbia = TrainingArguments(
    output_dir="./fine-tuned-medical-llama3-8B-lora",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    num_train_epochs=1,
    bf16=True,
    save_total_limit=3,
    push_to_hub=False,
    report_to="none"
)

trainer_columbia = Trainer(
    model=model_columbia,
    args=training_args_columbia,
    train_dataset=tokenized_train_columbia,
    eval_dataset=tokenized_eval_columbia,
    data_collator=data_collator_columbia
)

trainer_columbia.train()

model_columbia.save_pretrained("./fine-tuned-medical-llama3-8B-lora")
print("Fine-tuning complete and adapter saved.")



Request URL: https://api.fda.gov/drug/label.json?search=effective_time%3A%5B20241101+TO+20241115%5D+AND+version%3A1+AND+openfda.product_type%3Ahuman%2A&limit=100&skip=0
Status Code: 200
Request URL: https://api.fda.gov/drug/label.json?search=effective_time%3A%5B20241101+TO+20241115%5D+AND+version%3A1+AND+openfda.product_type%3Ahuman%2A&limit=100&skip=100
Status Code: 200
Request URL: https://api.fda.gov/drug/label.json?search=effective_time%3A%5B20241101+TO+20241115%5D+AND+version%3A1+AND+openfda.product_type%3Ahuman%2A&limit=100&skip=200
Status Code: 200
Request URL: https://api.fda.gov/drug/label.json?search=effective_time%3A%5B20241101+TO+20241115%5D+AND+version%3A1+AND+openfda.product_type%3Ahuman%2A&limit=100&skip=300
Status Code: 200
Request URL: https://api.fda.gov/drug/label.json?search=effective_time%3A%5B20241101+TO+20241115%5D+AND+version%3A1+AND+openfda.product_type%3Ahuman%2A&limit=100&skip=400
Status Code: 200
Request URL: https://api.fda.gov/drug/label.json?search=effect

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Map:   0%|          | 0/1010 [00:00<?, ? examples/s]

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
50,0.721300,0.729457


Fine-tuning complete and adapter saved.
